# Dubizzle used car sales data

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns",100)

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error,r2_score

from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("data.csv") #https://www.kaggle.com/datasets/alihassankp/dubizzle-used-car-sale-data

### EDA

In [3]:
df

,title,price_in_aed,kilometers,body_condition,mechanical_condition,seller_type,body_type,no_of_cylinders,transmission_type,regional_specs,horsepower,fuel_type,steering_side,year,color,emirate,motors_trim,company,model,date_posted
0,MITSUBISHI PAJERO 3.5L / 2013,"26,000",167390,Perfect inside and out,Perfect inside and out,Dealer,SUV,6,Automatic Transmission,GCC Specs,Unknown,Gasoline,Left Hand Side,2013.0,Silver,Dubai,GLS,mitsubishi,pajero,13/05/2022
1,chevrolet silverado,"110,000",39000,Perfect inside and out,Perfect inside and out,Dealer,SUV,8,Automatic Transmission,North American Specs,400 - 500 HP,Gasoline,Left Hand Side,2018.0,White,Sharjah,1500 High Country,chevrolet,silverado,14/01/2022
2,MERCEDES-BENZ E300 - 2014 - GCC SPEC - FULL OP...,"78,000",200000,Perfect inside and out,Perfect inside and out,Dealer,Sedan,6,Automatic Transmission,GCC Specs,400 - 500 HP,Gasoline,Left Hand Side,2014.0,Blue,Sharjah,E 300,mercedes-benz,e-class,05/05/2022
3,WARRANTY UNTIL APR 2023 || Ferrari 488 Spider ...,"899,000",27000,Perfect inside and out,Perfect inside and out,Dealer,Hard Top Convertible,8,Automatic Transmission,GCC Specs,600 - 700 HP,Gasoline,Left Hand Side,2018.0,Red,Dubai,Standard,ferrari,488-spider,30/04/2022
4,USED RENAULT DOKKER 2020,"33,000",69000,Perfect inside and out,Perfect inside and out,Owner,Wagon,4,Manual Transmission,GCC Specs,Less than 150 HP,Gasoline,Left Hand Side,2020.0,White,Dubai,Standard,renault,dokker,13/05/2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9965,AED7525month | 2015 Land Rover Range Rover Aut...,"215,000",105777,Perfect inside and out,Perfect inside and out,Dealer,SUV,8,Automatic Transmission,GCC Specs,500 - 600 HP,Gasoline,Left Hand Side,2015.0,White,Dubai,Autobiography,land-rover,range-rover,18/11/2021
9966,LOW MILEAGE Cadillac Escalade Platinum,"90,000",55640,Perfect inside and out,Perfect inside and out,Owner,SUV,8,Automatic Transmission,GCC Specs,400 - 500 HP,Gasoline,Left Hand Side,2014.0,White,Dubai,Platinum,cadillac,escalade,11/05/2022
9967,WARRANTY AVAILABLE || Range Rover Sport SVR Ca...,"679,000",100,Perfect inside and out,Perfect inside and out,Dealer,SUV,8,Automatic Transmission,Other,400 - 500 HP,Gasoline,Left Hand Side,NaN,Black,Dubai,SVR,land-rover,range-rover-sport,14/12/2021
9968,Chevrolet Malibu GCC Spec,"18,900",140000,"No accidents, very few faults",Perfect inside and out,Owner,Sedan,4,Automatic Transmission,GCC Specs,150 - 200 HP,Gasoline,Left Hand Side,2013.0,White,Dubai,LS,chevrolet,malibu,06/03/2022


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9970 entries, 0 to 9969
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 9965 non-null   object 
 1   price_in_aed          9970 non-null   object 
 2   kilometers            9970 non-null   int64  
 3   body_condition        9970 non-null   object 
 4   mechanical_condition  9970 non-null   object 
 5   seller_type           9970 non-null   object 
 6   body_type             9970 non-null   object 
 7   no_of_cylinders       9911 non-null   object 
 8   transmission_type     9970 non-null   object 
 9   regional_specs        9970 non-null   object 
 10  horsepower            9970 non-null   object 
 11  fuel_type             9970 non-null   object 
 12  steering_side         9970 non-null   object 
 13  year                  9000 non-null   float64
 14  color                 9970 non-null   object 
 15  emirate              

In [5]:
df.isnull().sum() #We examine the empty lines.

title                     5
price_in_aed              0
kilometers                0
body_condition            0
mechanical_condition      0
seller_type               0
body_type                 0
no_of_cylinders          59
transmission_type         0
regional_specs            0
horsepower                0
fuel_type                 0
steering_side             0
year                    970
color                     0
emirate                   0
motors_trim              28
company                   0
model                     0
date_posted               0
dtype: int64

### Feature Engineering

In [6]:
df["no_of_cylinders"].value_counts()

6          3325
4          2993
8          2986
12          366
Unknown      57
3            56
10           53
5            53
None         22
Name: no_of_cylinders, dtype: int64

In [7]:
df=df[df['no_of_cylinders']!="Unknown"]
df=df[df['no_of_cylinders']!="None"]
#We delete the Unknown and None values in no_of_cylinders.
df['no_of_cylinders'].fillna(6,inplace=True) #We fill the empty datas with 6.
df['no_of_cylinders']=df['no_of_cylinders'].astype(int) #We change the data type to integer.

In [8]:
df["price_in_aed"]=df.price_in_aed.str.replace(',','') #We delete the commas in price_in_aed.
df['price_in_aed']=df['price_in_aed'].astype(int)

In [9]:
df["motors_trim"].value_counts()

Other                2130
Standard              429
SE                    232
S                     227
G 63 AMG              197
                     ... 
XLE Plus                1
1500 2LT                1
Prestige Plus LWB       1
Limited + EL            1
2500HD LTZ              1
Name: motors_trim, Length: 855, dtype: int64

In [10]:
df['motors_trim'].fillna("Other",inplace=True) #We fill the empty datas in motors_trim with Other.

In [11]:
df["body_condition"].value_counts()

Perfect inside and out                9537
No accidents, very few faults          205
A bit of wear & tear, all repaired     118
Normal wear & tear, a few issues        31
Name: body_condition, dtype: int64

In [12]:
df["body_condition"]=df["body_condition"].map({"Perfect inside and out":0,"No accidents, very few faults":1,"A bit of wear & tear, all repaired":2,"Normal wear & tear, a few issues":3})
#We replace the data in body_condition with 0, 1, 2 and 3.
df['body_condition']=df['body_condition'].astype(int)

In [13]:
df["mechanical_condition"].value_counts()

Perfect inside and out              9745
Minor faults, all fixed              134
Major faults fixed, small remain       8
Ongoing minor & major faults           3
Major faults, all fixed                1
Name: mechanical_condition, dtype: int64

In [14]:
df["mechanical_condition"]=df["mechanical_condition"].map({"Minor faults, all fixed":0,"Major faults, all fixed":4,"Perfect inside and out":1,"Major faults fixed, small remain":2,"Ongoing minor & major faults":3})
df['mechanical_condition']=df['mechanical_condition'].astype(int)

In [15]:
df["seller_type"].value_counts()

Dealer                            6709
Owner                             2915
Dealership/Certified Pre-Owned     267
Name: seller_type, dtype: int64

In [16]:
df["seller_type"]=df["seller_type"].map({"Dealer":0,"Owner":1,"Dealership/Certified Pre-Owned":2})
df['seller_type']=df['seller_type'].astype(int)

In [17]:
df["body_type"].value_counts()

SUV                     4550
Sedan                   2726
Coupe                    845
Pick Up Truck            421
Hatchback                337
Sports Car               257
Hard Top Convertible     161
Van                      161
Soft Top Convertible     142
Crossover                107
Other                     97
Wagon                     66
Utility Truck             21
Name: body_type, dtype: int64

In [18]:
df["body_type"]=df["body_type"].map({"SUV":0,"Sedan":1,"Coupe":2,"Pick Up Truck":3,"Hatchback":4,"Sports Car":5,"Hard Top Convertible":6,"Van":7,"Soft Top Convertible":8,"Crossover":9,"Other":10,"Wagon":11,"Utility Truck":12})
df['body_type']=df['body_type'].astype(int)

In [19]:
df["transmission_type"].value_counts()

Automatic Transmission    9510
Manual Transmission        381
Name: transmission_type, dtype: int64

In [20]:
df["transmission_type"]=df["transmission_type"].map({"Automatic Transmission":0,"Manual Transmission":1})
df['transmission_type']=df['transmission_type'].astype(int)

In [21]:
df["regional_specs"].value_counts()

GCC Specs               7556
North American Specs    1144
Other                    526
European Specs           418
Japanese Specs           247
Name: regional_specs, dtype: int64

In [22]:
df["regional_specs"]=df["regional_specs"].map({"GCC Specs":0,"North American Specs":1,"Other":2,"European Specs":3,"Japanese Specs":4})
df['regional_specs']=df['regional_specs'].astype(int)

In [23]:
df["horsepower"].value_counts()

200 - 300 HP        2427
300 - 400 HP        2200
400 - 500 HP        1274
150 - 200 HP        1222
500 - 600 HP        1025
Unknown              768
600 - 700 HP         391
Less than 150 HP     387
700 - 800 HP         148
800 - 900 HP          30
900+ HP               19
Name: horsepower, dtype: int64

In [24]:
df=df[df['horsepower']!="Unknown"]
df["horsepower"]=df["horsepower"].map({"200 - 300 HP":0,"300 - 400 HP":1,"400 - 500 HP":2,"150 - 200 HP":3,"600 - 700 HP":4,"Less than 150 HP":5,"700 - 800 HP":6,"800 - 900 HP":7,"900+ HP":8,"500 - 600 HP":9})
df['horsepower']=df['horsepower'].astype(int)

In [25]:
df["fuel_type"].value_counts()

Gasoline    8925
Diesel       138
Electric      42
Hybrid        18
Name: fuel_type, dtype: int64

In [26]:
df["fuel_type"]=df["fuel_type"].map({"Gasoline":0,"Diesel":1,"Electric":2,"Hybrid":3})
df['fuel_type']=df['fuel_type'].astype(int)

In [27]:
df["steering_side"].value_counts()

Left Hand Side     9121
Right Hand Side       2
Name: steering_side, dtype: int64

In [28]:
df["steering_side"]=df["steering_side"].map({"Left Hand Side":0,"Right Hand Side":1})
df['steering_side']=df['steering_side'].astype(int)

In [29]:
df["color"].value_counts()

White          3039
Black          1922
Grey           1231
Silver          767
Blue            664
Red             555
Gold            227
Brown           218
Green           133
Orange           72
Beige            68
Yellow           68
Other Color      65
Burgundy         60
Purple           23
Teal              7
Tan               4
Name: color, dtype: int64

In [30]:
df["color"]=df["color"].map({"White":0,"Black":1,"Grey":2,"Silver":3,"Blue":4,"Red":5,"Gold":6,"Brown":7,"Green":8,"Orange":9,"Yellow":10,"Other Color":11,"Burgundy":12,"Purple":13,"Teal":14,"Tan":15,"Beige":16})
df['color']=df['color'].astype(int)

In [31]:
df['title'].fillna("None",inplace=True)
df=df.dropna() #We delete rows containing empty data.
df['year']=df['year'].astype(int)

In [32]:
df["emirate"]=df.emirate.str.replace(' ','')
df["emirate"].value_counts()

Dubai           6690
Sharjah          714
AbuDhabi         634
Ajman            126
AlAin             26
RasAlKhaimah      15
UmmAlQawain        6
Fujeirah           5
Name: emirate, dtype: int64

In [33]:
df["emirate"]=df["emirate"].map({"Dubai":0,"Sharjah":1,"AbuDhabi":2,"Ajman":3,"AlAin":4,"RasAlKhaimah":5,"UmmAlQawain":6,"Fujeirah":7})
df['emirate']=df['emirate'].astype(int)

In [34]:
df["model"].value_counts()

patrol               339
s-class              217
range-rover-sport    173
c-class              167
e-class              157
                    ... 
xk8                    1
lc-500                 1
pacifica               1
gs7                    1
odyssey-j              1
Name: model, Length: 518, dtype: int64

In [35]:
df["date_posted"]=pd.to_datetime(df["date_posted"])
df["day_posted"]=(df["date_posted"]).dt.day
df["month_posted"]=(df["date_posted"]).dt.month
df["year_posted"]=(df["date_posted"]).dt.year
df['day_posted']=df['day_posted'].astype(int)
df['month_posted']=df['month_posted'].astype(int)
df['month_posted']=df['month_posted'].astype(int)
del df["date_posted"]
df=df.reset_index()
del df["index"]
#We divide date_posted into day, month, year and time.

In [36]:
abs(df.corr()["price_in_aed"].sort_values(ascending=False)) #We look at their correlations.

price_in_aed            1.000000
no_of_cylinders         0.407235
horsepower              0.359233
year                    0.245268
regional_specs          0.179753
fuel_type               0.085201
body_type               0.083673
color                   0.070246
mechanical_condition    0.031142
day_posted              0.026800
steering_side           0.001606
kilometers              0.022247
transmission_type       0.031873
month_posted            0.031949
body_condition          0.055376
year_posted             0.055423
emirate                 0.094863
seller_type             0.104317
Name: price_in_aed, dtype: float64

### Regression

In [37]:
x,y=df.drop(["title","price_in_aed"],axis=1),df[["price_in_aed"]]
x.shape

(8216, 20)

In [38]:
x=pd.get_dummies(x,drop_first=True)
x=scaler.fit_transform(x)

In [39]:
def algo_test(x,y):
    L = LinearRegression()
    E = ElasticNet()
    R = Ridge()
    Lass = Lasso()
    ETR=ExtraTreeRegressor()
    GBR=GradientBoostingRegressor()
    XGBC= XGBRegressor()
    x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2,random_state=13)
    algos = [L,E,R,Lass,ETR,GBR,XGBC]
    algo_names = ['Linear','ElasticNet','Ridge','Lasso','Extra Tree','Gradient Boosting','XGradientBooting']
    r_squared = []
    rmse = []
    mae = []
    result = pd.DataFrame(columns = ['R_Squared','RMSE','MAE'],index = algo_names)
    for algo in algos:
        algo.fit(x_train,y_train)    
        r_squared.append(r2_score(y_test,algo.predict(x_test)))
        rmse.append(mean_squared_error(y_test, algo.predict(x_test))**.5)
        mae.append(mean_absolute_error(y_test, algo.predict(x_test)))
    result.R_Squared = r_squared
    result.RMSE = rmse
    result.MAE= mae
    return result.sort_values('R_Squared', ascending=False)

In [40]:
algo_test(x,y)

,R_Squared,RMSE,MAE
XGradientBooting,9.055256e-01,1.177717e+05,4.364382e+04
Gradient Boosting,8.676934e-01,1.393718e+05,6.399299e+04
Lasso,8.314037e-01,1.573288e+05,6.826429e+04
Extra Tree,8.269879e-01,1.593758e+05,5.119681e+04
Ridge,8.050559e-01,1.691761e+05,7.364734e+04
ElasticNet,1.351360e-01,3.563346e+05,1.741829e+05
Linear,-2.935072e+23,2.075838e+17,3.589110e+16


In [41]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [69]:
model=Sequential()
model.add(Dense(480,activation="relu"))
model.add(Dense(480,activation="relu"))
model.add(Dense(480,activation="relu"))
model.add(Dense(480,activation="relu"))
model.add(Dense(480,activation="relu"))
model.add(Dense(480,activation="relu"))
model.add(Dense(480,activation="relu"))
model.add(Dense(480,activation="relu"))
model.add(Dense(480,activation="relu"))
model.add(Dense(480,activation="relu"))
model.add(Dense(1))
model.compile(optimizer="adam",loss="mse")

In [70]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=128,epochs=150)
model.summary()

Epoch 1/150
52/52 [==============================] - 3s 23ms/step - loss: 153922404352.0000 - val_loss: 59139915776.0000
Epoch 2/150
52/52 [==============================] - 1s 20ms/step - loss: 83313967104.0000 - val_loss: 124536193024.0000
Epoch 3/150
52/52 [==============================] - 1s 20ms/step - loss: 83409436672.0000 - val_loss: 44523196416.0000
Epoch 4/150
52/52 [==============================] - 1s 20ms/step - loss: 74499989504.0000 - val_loss: 40298733568.0000
Epoch 5/150
52/52 [==============================] - 1s 20ms/step - loss: 90781278208.0000 - val_loss: 48781557760.0000
Epoch 6/150
52/52 [==============================] - 1s 19ms/step - loss: 55405592576.0000 - val_loss: 31395960832.0000
Epoch 7/150
52/52 [==============================] - 1s 20ms/step - loss: 50062012416.0000 - val_loss: 58328932352.0000
Epoch 8/150
52/52 [==============================] - 1s 20ms/step - loss: 39294398464.0000 - val_loss: 51495440384.0000
Epoch 9/150
52/52 [===================

Epoch 69/150
52/52 [==============================] - 1s 21ms/step - loss: 16342697984.0000 - val_loss: 29563709440.0000
Epoch 70/150
52/52 [==============================] - 1s 21ms/step - loss: 19624249344.0000 - val_loss: 23786340352.0000
Epoch 71/150
52/52 [==============================] - 1s 21ms/step - loss: 15482127360.0000 - val_loss: 26616666112.0000
Epoch 72/150
52/52 [==============================] - 1s 21ms/step - loss: 11547396096.0000 - val_loss: 25634093056.0000
Epoch 73/150
52/52 [==============================] - 1s 21ms/step - loss: 8125218816.0000 - val_loss: 24647974912.0000
Epoch 74/150
52/52 [==============================] - 1s 21ms/step - loss: 5919152128.0000 - val_loss: 25364422656.0000
Epoch 75/150
52/52 [==============================] - 1s 21ms/step - loss: 7991752704.0000 - val_loss: 23937579008.0000
Epoch 76/150
52/52 [==============================] - 1s 21ms/step - loss: 11151923200.0000 - val_loss: 27037810688.0000
Epoch 77/150
52/52 [===============

Epoch 137/150
52/52 [==============================] - 1s 21ms/step - loss: 8875980800.0000 - val_loss: 27941642240.0000
Epoch 138/150
52/52 [==============================] - 1s 21ms/step - loss: 4530452992.0000 - val_loss: 28846436352.0000
Epoch 139/150
52/52 [==============================] - 1s 21ms/step - loss: 2223518720.0000 - val_loss: 23886553088.0000
Epoch 140/150
52/52 [==============================] - 1s 21ms/step - loss: 1943375744.0000 - val_loss: 26943350784.0000
Epoch 141/150
52/52 [==============================] - 1s 21ms/step - loss: 1713904512.0000 - val_loss: 26282600448.0000
Epoch 142/150
52/52 [==============================] - 1s 21ms/step - loss: 1700322816.0000 - val_loss: 27859099648.0000
Epoch 143/150
52/52 [==============================] - 1s 21ms/step - loss: 1689756928.0000 - val_loss: 24917522432.0000
Epoch 144/150
52/52 [==============================] - 1s 21ms/step - loss: 1534380800.0000 - val_loss: 26498930688.0000
Epoch 145/150
52/52 [===========

In [71]:
tahmin=model.predict(x_test)

52/52 [==============================] - 0s 4ms/step


In [72]:
r2_score(tahmin,y_test)

0.8196356851066517

In [73]:
(mean_squared_error(tahmin,y_test))**0.5

163510.62620024834